<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/prueba_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prueba de modelos

In [5]:
from transformers import pipeline



In [6]:
data = ["hay que matarlos a todos", "Vamos a la plaza a marchar","Los chinos son todos sucios","Culpa de los bolitas"]

### Modelo 1 BETO binario

Parametros:

output_dir=repo_name,
eval_strategy ="steps",
adam_beta1=0.9,
adam_beta2=0.98,
eval_steps=10000,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
weight_decay=0.01,
push_to_hub=True,

In [7]:
specific_model = pipeline(model="natdebandi/beto-hate_speech_ar_bin")
specific_model(data)

[{'label': 'LABEL_1', 'score': 0.951015830039978},
 {'label': 'LABEL_0', 'score': 0.9810741543769836},
 {'label': 'LABEL_1', 'score': 0.9514239430427551},
 {'label': 'LABEL_1', 'score': 0.9515659809112549}]

### Modelo 2 BETO

Cambié los parámetros


In [8]:
specific_model = pipeline(model="natdebandi/beto-hate_speech_ar_bin2")
specific_model(data)

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/735k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9716325402259827},
 {'label': 'LABEL_0', 'score': 0.9867886304855347},
 {'label': 'LABEL_1', 'score': 0.97278892993927},
 {'label': 'LABEL_1', 'score': 0.9768657684326172}]